In [79]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os, sys

import os, json
from openai import OpenAI

client = OpenAI(
    # https://bailian.console.aliyun.com/?tab=model#/api-key
    api_key="sk-XX",

    # 大模型厂商的地址
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

completion = client.chat.completions.create(
    # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    model="qwen-plus", # 模型的代号

    messages=[
        {"role": "system", "content": "You are a helpful assistant."}, # 给大模型的命令，角色的定义
        {"role": "user", "content": "你是谁？"},  # 用户的提问
    ]
)
print(completion.choices[0].message.content)

我是通义千问，由阿里云研发的超大规模语言模型。我能够回答问题、创作文字，如写故事、公文、邮件、剧本等，还能进行逻辑推理、编程，甚至表达观点和玩游戏。我支持多种语言，包括中文、英文、德语、法语、西班牙语等。如果你有任何问题或需要帮助，欢迎随时告诉我！


In [126]:
TASK1_SUBMIT_DF = pd.read_csv("子赛题1：机组数据驱动的风电场短期风况预测/filtered_初赛_提交模板.csv")
agg_df = []

for keys, df in TASK1_SUBMIT_DF.groupby(["风场", "风机"]):
    paths = glob.glob("子赛题1：机组数据驱动的风电场短期风况预测/训练集/" + keys[0] + "/" + keys[1] + "/*")
    paths.sort()

    for path in paths[::-1]:
        df = pd.read_csv(path)
        df = df.dropna()
        if len(df) == 0:
            continue
        
        break

    result = json.dumps({
        "风向": df['风向'].tail(240).tolist(),
        "风速": df['风速'].tail(240).tolist()
    }, ensure_ascii=False)

    task1_prompt = f"""你是一个风电场短期风况预测专家，请结合下面电机的风向和风速，预测未来风向和风速未来下一个时刻的结果，只需要输出结果,输出结果为json格式，保留2位有效数字。
历史数据为{result}

接下来继续预测。
"""

    completion = client.chat.completions.create(
        # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
        model="qwen-flash", # 模型的代号
        messages=[
            {"role": "user", "content": task1_prompt},  # 用户的提问
        ]
    )
    result = json.loads(completion.choices[0].message.content)
    agg_df.append(list(keys) + [result["风向"], result["风速"]])

agg_df = pd.DataFrame(agg_df)
agg_df.columns = ["风场", "风机", "风向", "风速"]
pd.merge(TASK1_SUBMIT_DF.drop(["风向", "风速"], axis=1), agg_df, on=["风场", "风机"], how="left").to_csv("task1_sub.csv", index=None)

In [110]:
columns = ['SeqNum']
for i in range(1, 57):
    columns.append(f'Prediction{i}')

data = []
for i in range(1, 4):
    row = [i] + [0] * 56
    data.append(row)

TASK2_SUBMIT_DF = pd.DataFrame(data, columns=columns)



job1 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测01/入库流量数据.xlsx")
result = json.dumps({
    "Qi": job1['Qi'].tail(240).tolist(),
}, ensure_ascii=False)
task2_prompt = f"""你是一个水电站入库流量预测，流量数据，预测未来56时刻的结果，只需要输出结果,输出结果为json格式，以Qi为key，保留2位有效数字。
历史数据为{result}

接下来继续预测。
"""

completion = client.chat.completions.create(
    model="qwen-plus", # 模型的代号
    messages=[
        {"role": "user", "content": task2_prompt},  # 用户的提问
    ]
)
result = json.loads(completion.choices[0].message.content)
TASK2_SUBMIT_DF.loc[0] = [1] + result["Qi"]



job2 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测02/入库流量数据.xlsx")
result = json.dumps({
    "Qi": job2['Qi'].tail(240).tolist(),
}, ensure_ascii=False)
task2_prompt = f"""你是一个水电站入库流量预测，流量数据，预测未来56时刻的结果，只需要输出结果,输出结果为json格式，以Qi为key，保留2位有效数字。
历史数据为{result}

接下来继续预测。
"""

completion = client.chat.completions.create(
    model="qwen-plus", # 模型的代号
    messages=[
        {"role": "user", "content": task2_prompt},  # 用户的提问
    ]
)
result = json.loads(completion.choices[0].message.content)
TASK2_SUBMIT_DF.loc[1] = [2] + result["Qi"]



job3 = pd.read_excel("子赛题2：水电站入库流量预测/测试集_初赛/预测03/水库流量数据.xlsx")
result = json.dumps({
    "Qi": job3['Qi'].tail(240).tolist(),
}, ensure_ascii=False)
task2_prompt = f"""你是一个水电站入库流量预测，流量数据，预测未来56时刻的结果，只需要输出结果,输出结果为json格式，以Qi为key，保留2位有效数字。
历史数据为{result}

接下来继续预测。
"""

completion = client.chat.completions.create(
    model="qwen-plus", # 模型的代号
    messages=[
        {"role": "user", "content": task2_prompt},  # 用户的提问
    ]
)
result = json.loads(completion.choices[0].message.content)
TASK2_SUBMIT_DF.loc[2] = [3] + result["Qi"]
TASK2_SUBMIT_DF.to_csv("task2_sub.csv", index=None)

In [115]:
task_kucun = pd.read_csv("./子赛题3：离散制造行业产品物料需求智能预测/训练集/历史库存量.csv")
task_xuqiu = pd.read_csv("./子赛题3：离散制造行业产品物料需求智能预测/训练集/历史需求量.csv")
task_xuqiu["物料编码"] = task_xuqiu["物料编码"].astype(int)

sub = task_xuqiu.groupby(["工厂编码", "物料编码"])["需求量"].mean().reset_index()
sub.columns = ["工厂编码", "物料编码", "M+1月预测需求量"]
sub["M+2月预测需求量"] = sub["M+1月预测需求量"]
sub.to_csv("task3_sub.csv", index=None)